### Ch9_ChatbotAndLangchain_Databrick's Free Dolly with LangChain_Demo

In [2]:
!pip install "accelerate>=0.16.0,<1" "transformers[torch]>=4.28.1,<5" "torch>=1.13.1,<2"

!pip install langchain>=0.0.139

!pip install unstructured==0.7.12

  Obtaining dependency information for accelerate<1,>=0.16.0 from https://files.pythonhosted.org/packages/f7/fc/c55e5a2da345c9a24aa2e1e0f60eb2ca290b6a41be82da03a6d4baec4f99/accelerate-0.25.0-py3-none-any.whl.metadata
  Using cached accelerate-0.25.0-py3-none-any.whl.metadata (18 kB)
  Obtaining dependency information for transformers[torch]<5,>=4.28.1 from https://files.pythonhosted.org/packages/0f/12/d8e27a190ca67811f81deea3183b528d9169f10b74d827e0b9211520ecfa/transformers-4.36.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 1.7 MB/s eta 0:00:00a 0:00:01
  Using cached torch-1.13.1-cp310-cp310-manylinux1_x86_64.whl (887.5 MB)
  Obtaining dependency information for huggingface-hub from https://files.pythonhosted.org/packages/05/09/1945ca6ba3ad8ad6e2872ba682ce8d68c5e63c8e55458ed8ab4885709f1d/huggingface_hub-0.19.4-py3-none-any.whl.metadata
  Using cached huggingface_hub-0.19.4-py3-none-any.whl.metadata (14 kB)
  Obtaining dependency information fo

In [ ]:
# download CV data
!wget https://btcampdata.s3.amazonaws.com/gen-ai-data/Private-Data.zip
!unzip Private-Data.zip

--2023-12-11 16:20:22--  https://btcampdata.s3.amazonaws.com/gen-ai-data/Private-Data.zip
Resolving btcampdata.s3.amazonaws.com (btcampdata.s3.amazonaws.com)... 52.219.106.3, 52.219.106.180, 52.219.110.156, ...
Connecting to btcampdata.s3.amazonaws.com (btcampdata.s3.amazonaws.com)|52.219.106.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 548959 (536K) [application/zip]
Saving to: ‘Private-Data.zip.2’

Private-Data.zip.2  100%[===================>] 536.09K  2.67MB/s    in 0.2s    

2023-12-11 16:20:23 (2.67 MB/s) - ‘Private-Data.zip.2’ saved [548959/548959]

Archive:  Private-Data.zip
replace __MACOSX/._Private-Data? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
#!pip install unstructured

In [ ]:
#!pip install "unstructured[pdf]"

In [ ]:
import torch
from transformers import pipeline

generate_text = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16,
                         trust_remote_code=True, device_map="auto", return_full_text=True)


In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
import unstructured
from langchain.document_loaders import S3FileLoader
from langchain.document_loaders import UnstructuredFileLoader

In [ ]:
# template for an instrution with no input
prompt = PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}")

# template for an instruction with input
prompt_with_context = PromptTemplate(
    input_variables=["instruction", "context"],
    template="{instruction}\n\nInput:\n{context}")

hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

llm_chain = LLMChain(llm=hf_pipeline, prompt=prompt)
llm_context_chain = LLMChain(llm=hf_pipeline, prompt=prompt_with_context)


In [ ]:
#Loading pdf file as context to langchain
#  workspace  aws s3 ls s3://webage-genai-data/
                           #PRE Private-Data/

# loader = S3FileLoader("webage-genai-data", "Private-Data/CV1.pdf")

loader = UnstructuredFileLoader(
    "Private-Data/CV1.pdf")

data = loader.load()

In [ ]:
context = data[0].page_content
print(llm_context_chain.predict(instruction="Provide the career summary of CHRISTOPHOER MORGAN, senior web developer?", context=context).lstrip())

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("Private-Data/CV1.pdf")
context = loader.load_and_split()
context = context[0].page_content
context

In [ ]:
print(llm_context_chain.predict(instruction="What are the Certifications completed by CHRISTOPHOER MORGAN, senior web developer?",context=context).lstrip())

In [ ]:
print(llm_context_chain.predict(instruction="What is the phone number of CHRISTOPHOER MORGAN, senior web developer?", context=context).lstrip())

Let's see how well this performs when we load all the CV's at once

In [ ]:
context

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
loader = PyPDFDirectoryLoader("Private-Data")
data = loader.load()



In [ ]:
data